In [14]:
from keys import FTX_API_KEY, FTX_API_SECRET
from datetime import datetime
import time
import hmac
from requests import Request, Session, Response

class FtxClient:


    def __init__(self, api_key=None, api_secret=None, subaccount_name=None):
            self._session = Session()
            self._api_key = api_key
            self._api_secret = api_secret
            self._subaccount_name = subaccount_name
            self._ENDPOINT = 'https://ftx.com/api/'


    def sign_request(self, request: Request):
        ts = int(time.time() * 1000)
        request = Request('GET', self._ENDPOINT)
        prepared = request.prepare()
        print(prepared)
        signature_payload = f'{ts}{prepared.method}{prepared.path_url}'.encode()
        signature = hmac.new(self._api_secret.encode(), signature_payload, 'sha256').hexdigest()

        request.headers['FTX-KEY'] = self._api_key
        request.headers['FTX-SIGN'] = signature
        request.headers['FTX-TS'] = str(ts)

    def send_request(self, method: str, path: str, **kwargs):
        request = Request(method, self._ENDPOINT + path, **kwargs)
        self.sign_request(request)
        response = self._session.send(request.prepare())
        return self._process_response(response)

    def _process_response(self, response: Response):
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            if not data['success']:
                raise Exception(data['error'])
            return data['result']
    
    def get_all_trades(self, market: str, start_time: float = None, end_time: float = None) -> List:
        ids = set()
        limit = 100
        results = []
        while True:
            response = self._get(f'markets/{market}/trades', {
                'end_time': end_time,
                'start_time': start_time,
            })
            deduped_trades = [r for r in response if r['id'] not in ids]
            results.extend(deduped_trades)
            ids |= {r['id'] for r in deduped_trades}
            print(f'Adding {len(response)} trades with end time {end_time}')
            if len(response) == 0:
                break
            end_time = min(parse_datetime(t['time']) for t in response).timestamp()
            if len(response) < limit:
                break
        return results


In [49]:
dt = datetime(2020, 1, 1).timestamp()
acc = FtxClient(api_key=FTX_API_KEY, api_secret=FTX_API_SECRET)
res = acc.send_request('GET', 'markets/BTC-PERP/candles?resolution=15&limit=10')

print(res)

<PreparedRequest [GET]>
[{'close': 56810.0, 'high': 56813.0, 'low': 56804.0, 'open': 56804.0, 'startTime': '2021-03-12T04:53:00+00:00', 'time': 1615524780000.0, 'volume': 104218.2922}, {'close': 56827.0, 'high': 56827.0, 'low': 56810.0, 'open': 56810.0, 'startTime': '2021-03-12T04:53:15+00:00', 'time': 1615524795000.0, 'volume': 65427.5932}, {'close': 56820.0, 'high': 56835.0, 'low': 56814.0, 'open': 56827.0, 'startTime': '2021-03-12T04:53:30+00:00', 'time': 1615524810000.0, 'volume': 87843.6914}, {'close': 56817.0, 'high': 56831.0, 'low': 56815.0, 'open': 56820.0, 'startTime': '2021-03-12T04:53:45+00:00', 'time': 1615524825000.0, 'volume': 73576.3629}, {'close': 56830.0, 'high': 56830.0, 'low': 56817.0, 'open': 56817.0, 'startTime': '2021-03-12T04:54:00+00:00', 'time': 1615524840000.0, 'volume': 926.1438}, {'close': 56849.0, 'high': 56849.0, 'low': 56830.0, 'open': 56830.0, 'startTime': '2021-03-12T04:54:15+00:00', 'time': 1615524855000.0, 'volume': 134595.6102}, {'close': 56785.0, 'h

In [44]:
import requests
ENDPOINT='https://ftx.com/api'
res = requests.get(ENDPOINT+'markets/futures/BTC-PERP/candles?resolution={60}&limit={500}')
